In [2]:
import requests
import pandas as pd
import datetime
from sqlalchemy import create_engine, Column, Integer, String, Text, DateTime, Float, MetaData, Table
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import nltk
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


In [3]:
nltk.download('vader_lexicon')
analyzer = SentimentIntensityAnalyzer()


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/khaled/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [9]:
def fetch_news_articles(symbol, from_date, to_date, api_key):
    url = 'https://newsapi.org/v2/everything'
    query = f'"{symbol}" AND (stocks OR shares OR market)'
    params = {
        'q': query,
        'from': from_date,
        'to': to_date,
        'language': 'en',
        'sortBy': 'publishedAt',
        'apiKey': api_key,
        'pageSize': 100,
    }
    response = requests.get(url, params=params)
    data = response.json()
    if data.get('status') != 'ok':
        print(f"Error fetching news for {symbol}: {data.get('message')}")
        return []
    articles = data.get('articles', [])
    print(articles)
    for article in articles:
        article['symbol'] = symbol
    return articles


In [5]:
def compute_sentiment(text):
    if not text:
        return 0
    score = analyzer.polarity_scores(text)
    return score['compound']  # Range from -1 (negative) to 1 (positive)


In [11]:
NEWS_API_KEY = 'd243aae7a4c348b58f8df18be92a6c77'  # Replace with your actual API key

# Define date range
to_date = datetime.datetime.now()
from_date = to_date - datetime.timedelta(days=7)

# Format dates
to_date_str = to_date.strftime('%Y-%m-%dT%H:%M:%SZ')
from_date_str = from_date.strftime('%Y-%m-%dT%H:%M:%SZ')

all_articles = []

for symbol in ["NVDA","TSLA"]:
    print(f"Fetching news for {symbol}")
    articles = fetch_news_articles(symbol, from_date_str, to_date_str, NEWS_API_KEY)
    all_articles.extend(articles)


Fetching news for NVDA
[{'source': {'id': None, 'name': 'Biztoc.com'}, 'author': 'aol.com', 'title': 'Should You Forget Apple and Buy These 3 Tech Stocks Instead?', 'description': "Right now, Apple (NASDAQ: AAPL) is the world's largest company by market cap. However, investors need to be aware of some problems under the hood.\nInstead of Apple, I think there are much better buys right now, including Nvidia (NASDAQ: NVDA), Alphabet (NASDA…", 'url': 'https://biztoc.com/x/9e3077204ae13c7d', 'urlToImage': 'https://biztoc.com/cdn/9e3077204ae13c7d_s.webp', 'publishedAt': '2024-10-05T11:52:05Z', 'content': "Right now, Apple (NASDAQ: AAPL) is the world's largest company by market cap. However, investors need to be aware of some problems under the hood.Instead of Apple, I think there are much better buys … [+132 chars]"}, {'source': {'id': None, 'name': 'Yahoo Entertainment'}, 'author': 'Keith Speights, The Motley Fool', 'title': 'Could Nvidia Stock Soar if Kamala Harris Becomes President?', 'd

In [12]:
news_df = pd.DataFrame(all_articles)

# Keep only necessary columns
news_df = news_df[['publishedAt', 'symbol', 'title', 'description', 'content', 'url']]


In [14]:
# Convert 'publishedAt' to datetime
news_df['publishedAt'] = pd.to_datetime(news_df['publishedAt'])

# Remove duplicates based on URL
news_df.drop_duplicates(subset=['url'], inplace=True)

# Combine title and description for sentiment analysis
news_df['text'] = news_df['title'].fillna('') + ' ' + news_df['description'].fillna('')

# Compute sentiment score
news_df['sentiment'] = news_df['text'].apply(compute_sentiment)


In [15]:
news_df

,publishedAt,symbol,title,description,content,url,text,sentiment
0,2024-10-05 11:52:05+00:00,NVDA,Should You Forget Apple and Buy These 3 Tech S...,"Right now, Apple (NASDAQ: AAPL) is the world's...","Right now, Apple (NASDAQ: AAPL) is the world's...",https://biztoc.com/x/9e3077204ae13c7d,Should You Forget Apple and Buy These 3 Tech S...,-0.1779
1,2024-10-05 10:07:00+00:00,NVDA,Could Nvidia Stock Soar if Kamala Harris Becom...,Some of the vice president's proposals could b...,Nvidia's (NASDAQ: NVDA) share price has skyroc...,https://finance.yahoo.com/news/could-nvidia-st...,Could Nvidia Stock Soar if Kamala Harris Becom...,-0.0258
2,2024-10-05 08:51:01+00:00,NVDA,"Nvidia Stock Could Soar Another 561%, Accordin...",If you invested $100 in Nvidia (NASDAQ: NVDA) ...,If you invested $100 in Nvidia (NASDAQ: NVDA) ...,https://biztoc.com/x/fff39c5e670bbb2e,"Nvidia Stock Could Soar Another 561%, Accordin...",0.8957
3,2024-10-05 08:14:41+00:00,NVDA,NVIDIA Corporation (NVDA) Positioned for Growt...,We recently compiled a list of the 30 AI News ...,We recently compiled a list of the 30 AI News ...,https://finance.yahoo.com/news/nvidia-corporat...,NVIDIA Corporation (NVDA) Positioned for Growt...,0.6950
4,2024-10-04 23:25:16+00:00,NVDA,20 Most Powerful States in the US,"In this article, we will be taking a look at t...","In this article, we will be taking a look at t...",https://finance.yahoo.com/news/20-most-powerfu...,20 Most Powerful States in the US In this arti...,0.7897
...,...,...,...,...,...,...,...,...
195,2024-10-01 09:06:00+00:00,TSLA,Prediction: 3 Market-Leading Stocks That May P...,A victory for the former president may bode po...,"Exactly five weeks from today, on Nov. 5, vote...",https://finance.yahoo.com/news/prediction-3-ma...,Prediction: 3 Market-Leading Stocks That May P...,0.5719
196,2024-10-01 09:00:00+00:00,TSLA,Five Stocks to Watch in October—and What to Wa...,October brings a major product launch from one...,The major indexes are trading near record high...,https://www.investopedia.com/stocks-to-watch-o...,Five Stocks to Watch in October—and What to Wa...,0.5994
197,2024-10-01 06:28:15+00:00,TSLA,Are Tesla’s Golden Days Over?,The various functional failures of Tesla's Cyb...,The various functional failures of Tesla’s Cyb...,https://www.luxuo.com/motoring/automobile/are-...,Are Tesla’s Golden Days Over? The various func...,-0.5204
198,2024-10-01 05:40:53+00:00,TSLA,Tesla (TSLA) – Analysts’ Recent Ratings Updates,Tesla (NASDAQ: TSLA) recently received a numbe...,Tesla (NASDAQ: TSLA) recently received a numbe...,https://www.etfdailynews.com/2024/10/01/tesla-...,Tesla (TSLA) – Analysts’ Recent Ratings Update...,0.0772


In [17]:
# Database connection details
db_name = "stockdata"
db_user = "khaled"
db_password = "Arsenal4th-"
db_host = "localhost"
db_port = "5432"

# Create a connection engine
engine = create_engine(
    f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
)


In [18]:
Base = declarative_base()

class NewsArticle(Base):
    __tablename__ = 'news_articles'
    id = Column(Integer, primary_key=True)
    published_at = Column(DateTime)
    symbol = Column(String(10))
    title = Column(Text)
    description = Column(Text)
    content = Column(Text)
    url = Column(Text, unique=True)
    sentiment = Column(Float)
    created_at = Column(DateTime, default=datetime.datetime.now)


/tmp/ipykernel_81236/2941608782.py:1: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [19]:
Base.metadata.create_all(engine)

In [20]:
Session = sessionmaker(bind=engine)
session = Session()


In [21]:
from sqlalchemy.exc import IntegrityError

for index, row in news_df.iterrows():
    article = NewsArticle(
        published_at=row['publishedAt'],
        symbol=row['symbol'],
        title=row['title'],
        description=row['description'],
        content=row['content'],
        url=row['url'],
        sentiment=row['sentiment']
    )
    try:
        session.add(article)
        session.commit()
        print(f"Inserted article: {row['title']}")
    except IntegrityError:
        session.rollback()
        print(f"Duplicate article found, skipping: {row['url']}")
    except Exception as e:
        session.rollback()
        print(f"Error inserting article: {e}")


Error inserting article: (psycopg2.OperationalError) server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.

[SQL: INSERT INTO news_articles (published_at, symbol, title, description, content, url, sentiment, created_at) VALUES (%(published_at)s, %(symbol)s, %(title)s, %(description)s, %(content)s, %(url)s, %(sentiment)s, %(created_at)s) RETURNING news_articles.id]
[parameters: {'published_at': Timestamp('2024-10-05 11:52:05+0000', tz='UTC'), 'symbol': 'NVDA', 'title': 'Should You Forget Apple and Buy These 3 Tech Stocks Instead?', 'description': "Right now, Apple (NASDAQ: AAPL) is the world's largest company by market cap. However, investors need to be aware of some problems under the hood.\nInstead of Apple, I think there are much better buys right now, including Nvidia (NASDAQ: NVDA), Alphabet (NASDA…", 'content': "Right now, Apple (NASDAQ: AAPL) is the world's largest company by market cap. However, i